goal:
=====

basically a factorization machine with cross entropy loss with an additional "metric" kernal matrix and some non linearities + biases

In [1]:
# import this stuff
import time
import sys
from pylab import *
from scipy import sparse
import numpy as np

import tensorflow as tf

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

In [5]:
def factorize(observed_features,
              labels,
              observed_features_validation,
              labels_validation,
              rank,
              max_iter=100,
              verbose=False,
              lambda_v=0,
              lambda_k=0,
              lambda_w=0,
              lambda_constants=0,
              epsilon=0.001,
              optimizer=tf.train.AdamOptimizer(),
              seed=12345):

    # Extract info about shapes etc from the training data
    num_items = observed_features.shape[0]
    num_features = observed_features.shape[1]
 
    V = tf.Variable(tf.truncated_normal([rank, num_features], stddev=0.2, mean=0, seed=seed), name="feature_explainers")
    K = tf.Variable(tf.truncated_normal([rank, rank], stddev=0.2, mean=0, seed=seed), name="metric_matrix")
    
    w = tf.Variable(tf.truncated_normal([1, num_features], stddev=0.2, mean=0, seed=seed), name="hyperplane")
    b_one = tf.Variable(tf.truncated_normal([1, 1], stddev=0.2, mean=0, seed=seed), name="b_one")
    b_two = tf.Variable(tf.truncated_normal([rank, 1], stddev=0.2, mean=0, seed=seed), name="b_two")
   
    x = tf.placeholder(tf.float32, [None, num_features])
    y = tf.placeholder(tf.float32)
    
    norm_x = tf.nn.l2_normalize(x, dim=0)
    Vx = tf.tanh(tf.matmul(V, tf.transpose(norm_x)))
    right_kern = tf.matmul(K, Vx)
    
    full_kern = tf.matmul(tf.transpose(Vx), right_kern)
    linear = tf.matmul(w, tf.transpose(norm_x))

    pred = tf.reduce_sum(tf.sigmoid(linear + full_kern + b_one))
    
    cost = tf.reduce_mean(-y*tf.log(pred + 0.0000000001) - (1-y)*tf.log((1-pred + 0.0000000001)) + 
            lambda_v*tf.nn.l2_loss(V) +
            lambda_k*tf.nn.l2_loss(K) +
            lambda_w*tf.nn.l2_loss(w) +
            lambda_constants*(tf.nn.l2_loss(b_one) + tf.nn.l2_loss(b_two)))
    optimize = optimizer.minimize(cost)
    norm = tf.reduce_mean(tf.nn.l2_loss(V))
    
    init = tf.initialize_all_variables()
    
    with tf.Session() as sess:
        sess.run(init)
        last_cost = 1000000
        for iter in range(0, max_iter):
            avg_cost = 0
            
            for i in range(num_items):
                _, c, n = sess.run([optimize, cost, norm],
                              feed_dict={x:observed_features[i].reshape(1, num_features), y:labels[i]})
                avg_cost += c / num_items
            if verbose:
                print("epoch: %s, cost: %s" % (iter+1, avg_cost))

            # check for convergence
            if abs(avg_cost-last_cost)/avg_cost < epsilon:
                break
                
            last_cost = avg_cost
            
        if verbose:
            print("optimization finished")
        predictions = []
        total_costs = 0
        for i in range(observed_features_validation.shape[0]):
            p, c = sess.run([pred, cost], feed_dict={x:observed_features_validation[i].reshape(1, num_features), y:labels_validation[i]})
            predictions.append(p)
            total_costs += c
        return predictions, total_costs/observed_features_validation.shape[0], sess.run([norm])

In [6]:
# use this data for now

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

ng = datasets.fetch_20newsgroups (categories=categories, shuffle=True)
labels = [1 if y == 2 else 0 for y in ng.target.reshape(-1,1)]

tfidf = TfidfVectorizer(decode_error=False, min_df=5)

X_train, X_test, y_train, y_test = train_test_split(ng.data, labels, test_size=.3)
X_train = tfidf.fit_transform(X_train).todense()
X_test = tfidf.transform(X_test).todense()


In [7]:
r = 10
predictions, test_costs, norm = factorize(X_train, y_train, X_test, y_test, r, verbose=True, lambda_v=0.01, max_iter=300)
print("rank: %s, cost: %s, overall AUC: %s, norm: %s") % (r, test_costs, roc_auc_score(y_test, predictions, average="weighted"), norm)

epoch: 1, cost: 1.91015329636
epoch: 2, cost: 0.307251415972
epoch: 3, cost: 0.113321662633
epoch: 4, cost: 0.0783280579537
epoch: 5, cost: 0.0601871775002
epoch: 6, cost: 0.049063640731
epoch: 7, cost: 0.0326555883105
epoch: 8, cost: 0.023102213897
epoch: 9, cost: 0.0269521611122
epoch: 10, cost: 0.0278797101783
epoch: 11, cost: 0.0137090436419
epoch: 12, cost: 0.0111679183181
epoch: 13, cost: 0.00930694758596
epoch: 14, cost: 0.00621116050623
epoch: 15, cost: 0.00514938197495
epoch: 16, cost: 0.00271649706442
epoch: 17, cost: 0.00153523851702
epoch: 18, cost: 0.000915212788428
epoch: 19, cost: 0.000632242187385
epoch: 20, cost: 0.000453419271512
epoch: 21, cost: 0.00032883057859
epoch: 22, cost: 0.000240453385434
epoch: 23, cost: 0.000176941198825
epoch: 24, cost: 0.00013102228579
epoch: 25, cost: 9.75193899209e-05
epoch: 26, cost: 7.29390050157e-05
epoch: 27, cost: 5.4766117771e-05
epoch: 28, cost: 4.12939196065e-05
epoch: 29, cost: 3.12327421895e-05
epoch: 30, cost: 2.37113803998e-

## 